In [1]:
from importlib import reload
from itertools import repeat
from date_utils import partition_date_range
import pandas as pd

In [2]:
import functions as fu
reload(fu)

/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_geopandas/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


<module 'functions' from '/home/maximilian.hofmann/tools/brand_science_pricing/data/functions.py'>

In [11]:
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count

In [6]:
with open('/home/maximilian.hofmann/tools/brand_science_pricing/data/queries/sql_otter_data.sql', 'r') as f:
    sql_template = f.read()

In [7]:
project_name = 'css-operations'
stag_dataset_name = 'brand_science_stag'
stag_table_name = 'pricing_otter_data_stag'
full_stag_table_name = '.'.join([project_name, stag_dataset_name, stag_table_name])

In [8]:
begin_date = '2022-06-01'
end_date = '2022-09-30'
days_per_batch = 10

In [9]:
date_list = partition_date_range(pd.to_datetime(begin_date), pd.to_datetime(end_date), days_per_batch)
date_list

[['2022-06-01', '2022-06-20'],
 ['2022-06-21', '2022-07-10'],
 ['2022-07-11', '2022-07-30'],
 ['2022-07-31', '2022-08-19'],
 ['2022-08-20', '2022-09-08'],
 ['2022-09-09', '2022-09-28'],
 ['2022-09-29', '2022-09-30']]

In [12]:
country_code = 'CO'
clustering_fields = ['country_code']

In [14]:
res = []
with Pool(min(cpu_count(), len(date_list))) as p:
    r = p.starmap(fu.process_otter_data_date_range, zip(repeat(sql_template), date_list, repeat(country_code), repeat(full_stag_table_name), repeat(clustering_fields)))
    res.append(r)

Start data cleaning...
Data cleaning finished!
Start data cleaning...
Data cleaning finished!
Start data cleaning...
Data cleaning finished!
Start data cleaning...
Data cleaning finished!
Start data cleaning...
Data cleaning finished!
Start data cleaning...
Data cleaning finished!
Start data cleaning...
Data cleaning finished!


In [ ]:
prod_dataset_name = 'brand_science'
prod_table_name = 'pricing_otter_data'
full_prod_table_name = '.'.join([project_name, prod_dataset_name, prod_table_name])

In [ ]:
create_tbl = fu.create_tbl(full_stag_table_name, full_prod_table_name)

In [ ]:
upsert_tbl = fu.upsert_tbl_otter_data(full_stag_table_name, full_prod_table_name)